In [1]:
from __future__ import (absolute_import, division, print_function,unicode_literals)

import yfinance as yf
import pandas as pd
import backtrader as bt
from datetime import datetime
import os
import sys
import json
from backtrader.analyzers import Returns,DrawDown,SharpeRatio,TradeAnalyzer

In [4]:

# import strategies
sys.path.append('../backtrader-testing/')
from  strategies import TestStrategy

In [5]:
def prepare_cerebro(asset,strategy,data_path,start_date:str,end_date:str=datetime.now(),cash:int=100,commission:float=0)->bt.Cerebro:
    if end_date ==None:
        end_date= datetime.strftime(datetime.now(),"%Y-%m-%d")
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(cash)
    cerebro.broker.setcommission(commission=commission)
    cerebro.addstrategy(strategy)
    
    isExist = os.path.exists(data_path)
    if not isExist:
        data= yf.download(asset,start_date,end=end_date)
        data.to_csv(data_path)
    data = bt.feeds.YahooFinanceCSVData(
        dataname=data_path,
        fromdate=datetime.strptime(start_date,"%Y-%m-%d"),
        todate=datetime.strptime(end_date,"%Y-%m-%d"),
        reverse=False
    )
    cerebro.adddata(data)
    cerebro.addanalyzer(TradeAnalyzer)
    return cerebro

In [7]:
def run_test(cerebro:bt.Cerebro):

    result={}
    cerebro.addanalyzer(SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(Returns, _name='returns')
    cerebro.addanalyzer(DrawDown, _name='draw')
    cerebro.addanalyzer(TradeAnalyzer, _name='trade')
    
    starting = cerebro.broker.getvalue()
    res=cerebro.run()
    final=cerebro.broker.getvalue()

    thestrat = res[0]

    sharpe=thestrat.analyzers.sharpe.get_analysis()
    return_val=thestrat.analyzers.returns.get_analysis()
    drawdown=thestrat.analyzers.draw.get_analysis()
    trade=thestrat.analyzers.trade.get_analysis()

    result["sharpe_ratio"]=sharpe['sharperatio']
    result["return"]=return_val['rtot']
    result['max_drawdown'] = drawdown['max']['drawdown']
    
    try:
        result['win_trade']=trade['won']['total']
    except:
        result['win_trade']="Undefined"
    
    try:
        result['loss_trade']=trade['lost']['total']
    except:
        result['loss_trade']="Undefined"

    try:
        result['total_trade']=trade['total']['total']
    except:
        result['total_trade']="Undefined"

    result['start_portfolio']=starting
    result['final_portfolio']=final

    return result

In [8]:
cerebro=prepare_cerebro('ALGO',TestStrategy,"ALGO-USD.csv","2021-10-01","2022-10-19")

result=run_test(cerebro)
result

2021-10-01, Close, 99.18
2021-10-04, Close, 99.15
2021-10-05, Close, 99.07
2021-10-05, BUY CREATE, 99.07
2021-10-06, BUY EXECUTED, 99.06
2021-10-06, Close, 99.06
2021-10-07, Close, 98.92
2021-10-08, Close, 98.86
2021-10-11, Close, 98.72
2021-10-12, Close, 98.82
2021-10-13, Close, 98.84
2021-10-13, SELL CREATE, 98.84
2021-10-14, SELL EXECUTED, 98.94
2021-10-14, Close, 98.94
2021-10-15, Close, 98.78
2021-10-18, Close, 98.67
2021-10-18, BUY CREATE, 98.67
2021-10-19, BUY EXECUTED, 98.58
2021-10-19, Close, 98.60
2021-10-20, Close, 98.67
2021-10-21, Close, 98.47
2021-10-22, Close, 98.51
2021-10-25, Close, 98.57
2021-10-26, Close, 98.63
2021-10-26, SELL CREATE, 98.63
2021-10-27, SELL EXECUTED, 98.77
2021-10-27, Close, 98.77
2021-10-28, Close, 98.65
2021-10-29, Close, 98.68
2021-11-01, Close, 98.62
2021-11-02, Close, 98.75
2021-11-03, Close, 98.69
2021-11-04, Close, 98.91
2021-11-05, Close, 99.11
2021-11-08, Close, 98.93
2021-11-09, Close, 99.06
2021-11-10, Close, 98.70
2021-11-11, Close, 98.5

{'sharpe_ratio': -1.2322219739407292,
 'return': -0.03822121282019809,
 'max_drawdown': 4.295515561300581,
 'win_trade': 9,
 'loss_trade': 15,
 'total_trade': 25,
 'start_portfolio': 100,
 'final_portfolio': 96.24999999999997}